In [41]:
! pip install conllu
! python -m spacy download ru_core_news_sm -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 45.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [42]:
import spacy

# загружаем русскую модель
nlp = spacy.load('ru_core_news_sm')

In [43]:
from io import open
from conllu import parse_incr

# читаем коннлу файл
data_file = open("conllu_stories.conllu", "r", encoding="utf-8")
sent_list = [tokenlist for tokenlist in parse_incr(data_file)]

In [70]:
# функция, которая парсит запрос пользователя и возвращает список условий для поиска
def parse_query(query):
    conditions = []
    # делим по пробелам
    query_parts = query.split()
    # для каждой части запроса
    for part in query_parts:
        # если запрос в кавычках
        if part.startswith('"') and part.endswith('"'):
            # достаём и записываем как словоформу
            form = part.strip('"')
            conditions.append({"form": form})
        # если все буквы заглавные
        elif part.isupper():
            # то это часть речи
            upos = part
            conditions.append({"upos": upos})
        # если в строке есть +
        elif "+" in part:
            # то лемма и часть речи
            lemma, upos = part.split("+")
            conditions.append({"lemma": lemma, "upos": upos})
        # в ином случае
        else:
            # находим лемму введённого слова и записываем её
            lemma = nlp(part)[0].lemma_
            conditions.append({"lemma": lemma})
    return conditions  # возвращаем условия

# собственно функция поиска
def search_conllu_by_chain(conllu_text, query):
    # определяем необходимые условия запроса
    conditions = parse_query(query)
    results = []

    # проходимся по предложениям
    for sentence in conllu_text:
      # достаём название страшилки, id предложения и текст предложения из метаданных
      text_title, sent_id, sent_text = list(sentence.metadata.values())

      # итерируемся по последовательным токенам в зависимости от длины цепочки
      for i in range(len(sentence) - len(conditions) + 1):
          match = True
          for j, condition in enumerate(conditions):
              token = sentence[i + j]

              # проверяем на соответствие каждому из требуемых условий
              if "lemma" in condition and token.get("lemma") != condition["lemma"]:
                  # если несоответствует - прерываем
                  match = False
                  break
              if "upos" in condition and token.get("upostag") != condition["upos"]:
                  match = False
                  break
              if "form" in condition and token.get("form") != condition["form"]:
                  match = False
                  break

          # если все условия соблдюдены
          if match:
              # сохраняем форму, лемму и pos-тег
              matched_tokens = [(token.get("form"), token.get("lemma"), token.get("upostag")) for token in sentence[i:i + len(conditions)]]
              # и добавляем к результатам название, id и текст предложения
              results.append((text_title, sent_id, sent_text, matched_tokens))

    return results

In [89]:
# запрос пользователя
query = 'прийти NUM NOUN'

In [90]:
# ищем цепочку по запросу
results = search_conllu_by_chain(sent_list, query)

# печатаем результаты
print(f"Результаты поиска по запросу '{query}':")
for text_title, sent_id, sent_text, matched_tokens in results:
    matched_str = " -> ".join([f'"{form}" ({lemma}, {upos})' for form, lemma, upos in matched_tokens])
    print(f'В страшилке "{text_title}" нашлось {matched_str}: \n{sent_text}')

Результаты поиска по запросу 'прийти NUM NOUN':
В страшилке "История про Барби" нашлось "пришла" (прийти, VERB) -> "одна" (одна, NUM) -> "женщина" (женщина, NOUN): 
И к ним пришла одна женщина с дочкой, ну, их подруга.
В страшилке "Красный бантик" нашлось "Пришли" (прийти, VERB) -> "два" (два, NUM) -> "милиционера" (милиционер, NOUN): 
Пришли два милиционера и остались.
